# Balancer Simulations Math Challenge - Basic Exercises

This is a series of exercises to gain intuition for the core algorithm in Balancer Pools: the Value Function, and invariant V.

**Exercise:** We'll work on Step 1 questions in our working session on May 25 in teams of 2, facilitated by Octopus and Angela.  
**Challenges:** Step 2 will be homework assignments, we'll offer a range challenges, XXX

## Exercise:   
Let's set up a pool.  

We have 100 Token A and 100 Token B, with equal weights.  
The price definition in our pool is constraint by the
Invariant V in Balancer Pools. 

a_bal = balance of token A  
b_bal = balance of token B  
a_weight = weight of token A  
b_weight = weight of token B   

The weights in this pool are equal for both tokens.
For now, we don't have a swap fee.

# Q1.1: 
What's the initial spot price of token A in token B?

In [ ]:
a_bal = 100
b_bal = 100

a_weight = 0.5
b_weight = 0.5

In [ ]:
spot_price = (a_bal/a_weight)/(b_bal/b_weight)
print(spot_price)

# Q1.2:
Now let's assume a series of 99 swaps. With every swap, **1.0 token A is bought from the pool, against token B**.  

**a) Create a table "buy_A"** with

    the token A balances (swap by swap)
    the token B balances (swap by swap) - that are constraint by the value function. 

**b) What do you notice in general?**
Write down your findings (in words).

**c) How much would Alice have to pay in token B when buying the first 1.0 token A?**
Write down your findings (in words). Compare with the initial Spotprice.

In [ ]:
import pandas as pd

In [ ]:
a_bal = 100
b_bal = 100

a_weight = 0.5
b_weight = 0.5

In [ ]:
a = [100 - i for i in range(100)]
b = [100 * 100 / i for i in a]

pool = {
    'a': a,
    'b': b
}

In [ ]:
pool_df = pd.DataFrame.from_dict(pool)
pool_df

In [ ]:
a_bal = 100
b_bal = 100
a_weight = 0.5
b_weight = 0.5
s_f = 0.0 #swap fee

inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant
a_vals = pd.Series(range(100,0,-1)) #create df with based on a_vals

buy_A = pd.DataFrame(a_vals, columns=['token_A'])
buy_A['invariant'] = inv #value required to calculate token B value

#create values for plot, add Y_balances according to current invariant
buy_A['token_B'] = (buy_A.invariant/(buy_A.token_A**a_weight))**(1/b_weight)

buy_A.head(10)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline

In [ ]:
plt.plot(buy_A.head(60)['token_A'], buy_A.head(60)['token_B'])
plt.xlabel("token_A")
plt.ylabel("token_B")
plt.show()

## Curve-ball Question #1
Which transaction(s) ends up cheaper?
* Purchasing 99 token_A's in 99 seperate even transactions.
* Purchasing 99 token_A's in a single transaction.

In [ ]:
# Purchasing 99 tokens in 99 seperate even transactions:
seperate_total_cost = buy_A[buy_A['token_A'] == 1].iloc[0]['token_B'] - buy_A[buy_A['token_A'] == 100].iloc[0]['token_B']
print(seperate_total_cost)

In [ ]:
# Purchasing 99 tokens in 1 transaction:
bulk_total_cost = 100*((100/(100-99))**(0.5/0.5)-1)
print(bulk_total_cost)

It appears that both paths appear equivalent in terms of price of exchange.

## Curve-ball Question #2
What about if we factor in pool fees > 0% where fees get immediately added to the pool asset balance(s)?

Assumptions:
* A 0.30% swap fee
* Invariant value changes post-swap with the addition of swap fee(s).
* Taking the calc_given_in function outlined in [balancer_math.py](https://token-engineering-balancer.gitbook.io/balancer-simulations/additional-code-and-instructions/balancer-the-python-edition/balancer_math.py)

In [ ]:
swap_fee = 0.003

In [ ]:
# Purchasing 99 tokens in 99 seperate even transactions:
a_bal = 100
b_bal = 100
a_weight = 0.5
b_weight = 0.5
s_f = 0.003 #swap fee

inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant
a_vals = pd.Series(range(100,0,-1)) #create df with based on a_vals

buy_A_with_fees = pd.DataFrame(a_vals, columns=['token_A'])

token_B_list = [100]
inv_list = [inv]
for a_bal in buy_A['token_A']:
    # skip a_bal=b_bal=100 and 
    if a_bal == 100: continue
    
    a_bal = a_bal+1
    b_bal = token_B_list[-1]
    
    amount_in_B = b_bal*((a_bal/(a_bal-1))**(a_weight/b_weight)-1)*(1/(1-s_f))
    new_b_bal = token_B_list[-1]+amount_in_B
    
    token_B_list.append(new_b_bal)
    
    inv = ((a_bal-1)**a_weight)*(new_b_bal**b_weight) #calculate invariant
    # inv = 100
    inv_list.append(inv)

buy_A_with_fees['invariant'] = inv_list
buy_A_with_fees['token_B'] = token_B_list
print(buy_A_with_fees)

In [ ]:
# Result
seperate_total_cost = buy_A_with_fees[buy_A_with_fees['token_A'] == 1].iloc[0]['token_B'] - buy_A_with_fees[buy_A_with_fees['token_A'] == 100].iloc[0]['token_B']
print(seperate_total_cost)

In [ ]:
# Purchasing 99 tokens in 1 transactions:
bulk_total_cost = 100*((100/(100-99))**(0.5/0.5)-1)*(1/(1-swap_fee))
print(bulk_total_cost)

In [ ]:
# Difference
print("Difference seperate/bulk: "+str(seperate_total_cost-bulk_total_cost))
print("Ratio seperate/bulk: "+str(seperate_total_cost/bulk_total_cost))

In [ ]:
# Test:
bulk_total_cost = 100*((100/(100-1))**(0.5/0.5)-1)*(1/(1-swap_fee))
print(bulk_total_cost)

In [ ]:
from matplotlib.legend_handler import HandlerLine2D

plt.figure(figsize=(12, 12))
line1, = plt.plot(buy_A['token_A'], buy_A['token_B'], color='green', label="no fees")
line2, = plt.plot(buy_A_with_fees['token_A'], buy_A_with_fees['token_B'], color='blue', label="with fees")

plt.legend(handler_map={line1: HandlerLine2D(numpoints=4)})

plt.xlabel("token_A")
plt.ylabel("token_B")


plt.show()

# Q1.3:

Now let's assume a series of 99 swaps in the opposite direction. We start again with the original state: We have 100 Token A and 100 Token B.   
With every swap, **1.0 token B is bought from the pool, against token A**.

Create a table **'buy_B'** with

    the token A balances (swap by swap)
    the token B balances (swap by swap) - that are constraint by the value function.

In [ ]:
a_bal = 100
b_bal = 100
a_weight = 0.5
b_weight = 0.5
s_f = 0.0 #swap fee

inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant
b_vals = pd.Series(range(100,0,-1)) #create df with based on b_vals

buy_B = pd.DataFrame(b_vals, columns=['token_B'])
buy_B['invariant'] = inv #value required to calculate token A value

#create values for plot, add Y_balances according to current invariant
buy_B['token_A'] = (buy_B.invariant/(buy_B.token_B**b_weight))**(1/a_weight)

buy_B.tail(10)

# Q1.4:
What is the new price of token A in token B after 90 swaps token A for B?

In [ ]:
# Where balance of token_B = 100 - 90 = 10:
buy_B.loc[buy_B['token_B']<= 10]

In [ ]:
# Spot Price
spot_price = buy_B.loc[buy_B['token_B']== 10]['token_A'] / buy_B.loc[buy_B['token_B']== 10]['token_B']
print(spot_price)

# Q1.5:  
Now create a graph (use plotly or similar), and draw the full curve for this series of both kinds of swaps - the AMM curve.

In [ ]:
buy_A.head(5)

In [ ]:
buy_B.head(5)

In [ ]:
combined = pd.concat([buy_A,buy_B[buy_B['token_B'] < 100]])

In [ ]:
combined = combined.sort_values(by=['token_B'])

In [ ]:
combined_p = combined[(combined['token_A'] <= 500) & (combined['token_B'] <= 500)]

In [ ]:
combined_p

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

%matplotlib inline

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(combined_p['token_A'], combined_p['token_B'])
plt.xlabel("token_A")
plt.ylabel("token_B")


plt.show()

# Q1.6:
Take this plot, and mark 
- the initial price in Q1.1 (starting price)
- the new price in Q1.4 (after 90 swaps)

In [ ]:
# Initial price where balance of token_A = 100:
print(combined.loc[combined['token_B']== 100])
print("Spot price at this point: " + str(1))

# Where balance of token_B = 100 - 90 = 10:
print(combined.loc[combined['token_B']== 10])
print("Spot price at this point: " + str(100))

# Q1.7:
Formulate a "rule of a thumb", how do swaps effect the price?

### As the proportion of tokens in the pool get more and more offset, the more drastic the 'slippage' for making a unit trade in the pool becomes.

In [ ]:
combined['spot_price_a'] = combined['token_B'] / combined['token_A']

combined_p = combined[(combined['token_A'] <= 500) & (combined['token_B'] <= 500)]

plt.figure(figsize=(6, 6))
plt.plot(combined_p['token_A'], combined_p['spot_price_a'])
plt.xlabel("token_A")
plt.ylabel("spot_price_a")


plt.show()

# Now, let's consider weights!

We continue with the value function $V = a^w_a*b^w_b  $
where  
$a$ = balance of token asset A  
$b$ = balancer of token asset B  
$w_a$ = weight of token asset A  
$w_b$ = weight of token asset B  

# Q2.1:  
Write down the value function for the pool in Q1.1!

In [ ]:
a_bal = 100
b_bal = 100

a_weight = 0.5
b_weight = 0.5

In [ ]:
V = (a_bal**a_weight)*(b_bal**b_weight)
print(V)

# Q2.2:
Let's got back to your initial balances in Step 1 in the pool:  
100 tokens A  
100 tokens B  

How do you need to change the weights in order to land at a **price of  
4 tokens A : 1 token B**


$ SP = \frac{\frac{B_i}{W_i}}{\frac{B_o}{W_o}} $

Where we know $ SP = 4, B_i = 100, B_o = 100 $

We solve for $ W_i $ and $ W_o $

$W_i+W_o=1$

$4=(\frac{W_o}{W_i})$

$W_o = 4*W_i$

$W_o+W_i = 1$

$W_o=1-W_i$

$ 1-W_i = 4*W_i$

$5*W_i = 1$

Therefore,

$W_i = 0.2$

$W_o = 0.8$




In [ ]:
a_weight = 0.2
b_weight = 0.8

spot_price = (a_bal/a_weight)/(b_bal/b_weight)
print(spot_price)

Provide the new value function!

In [ ]:
V = (a_bal**a_weight)*(b_bal**b_weight)
print(V)

# Q2.3:
Create a graph showing the new AMM Curve in Q2.2  
Compare to the graph in Q1.4 - how does a change in weights change the graph?

In [ ]:
a_bal = 100
b_bal = 100
a_weight = 0.2
b_weight = 0.8
s_f = 0.0 #swap fee

inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant
a_vals = pd.Series(range(100,0,-1)) #create df with based on a_vals

buy_A = pd.DataFrame(a_vals, columns=['token_A'])
buy_A['invariant'] = inv #value required to calculate token B value

#create values for plot, add Y_balances according to current invariant
buy_A['token_B'] = (buy_A.invariant/(buy_A.token_A**a_weight))**(1/b_weight)

In [ ]:
a_bal = 100
b_bal = 100
a_weight = 0.2
b_weight = 0.8
s_f = 0.0 #swap fee

inv = (a_bal**a_weight)*(b_bal**b_weight) #calculate invariant
b_vals = pd.Series(range(100,0,-1)) #create df with based on b_vals

buy_B = pd.DataFrame(b_vals, columns=['token_B'])
buy_B['invariant'] = inv #value required to calculate token A value

#create values for plot, add Y_balances according to current invariant
buy_B['token_A'] = (buy_B.invariant/(buy_B.token_B**b_weight))**(1/a_weight)

In [ ]:
w_combined = pd.concat([buy_A,buy_B[buy_B['token_B'] < 100]])

In [ ]:
w_combined = w_combined.sort_values(by=['token_B'])

In [ ]:
w_combined_p = w_combined[(w_combined['token_A'] <= 500) & (w_combined['token_B'] <= 500)]

In [ ]:
from matplotlib.legend_handler import HandlerLine2D

plt.figure(figsize=(6, 6))
line1, = plt.plot(w_combined_p['token_A'], w_combined_p['token_B'], color='green', label="20:80")
line2, = plt.plot(combined_p['token_A'], combined_p['token_B'], color='blue', label="50:50")

plt.legend(handler_map={line1: HandlerLine2D(numpoints=4)})

plt.xlabel("token_A")
plt.ylabel("token_B")


plt.show()

# Q2.4:
Compare token prices in this pool.  
How much would Alice have to pay in case there are only 2 tokens left in the pool  
    **a) buy 1.0 token A for token B**  
    **b) buy 1.0 token B for token A**   

In [ ]:
w_combined_p[(w_combined_p['token_A'] >= 1.0) & (w_combined_p['token_A'] <= 3.0)]

In [ ]:
# Spot Price
inst_a_bal = 2.0
inst_b_bal = buy_A.loc[buy_A['token_A']== 2]['token_B']
a_spot_price = inst_b_bal / inst_a_bal

inst_b_bal_after_trade = (inv/((inst_a_bal - 1)**a_weight))**(1/b_weight)
a_effective_price = (inst_b_bal_after_trade - inst_b_bal)
print(a_spot_price)
print(a_effective_price)